In [ ]:
import zipfile
import pandas as pd
import os
import numpy as np
# Diretório onde estão os arquivos .zip
zip_dir = 'C:\\Users\\msses\\Desktop\\ETF'

# Diretório temporário para extração
extract_dir = 'extracted_files'

# Lista para armazenar todos os dataframes
df_list = []

# Itera por todos os arquivos .zip na pasta
for file in os.listdir(zip_dir):
    if file.endswith('.zip'):
        zip_path = os.path.join(zip_dir, file)
        
        # Extrai os arquivos do .zip
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        
        # Pega todos os .txt extraídos
        txt_files = [f for f in os.listdir(extract_dir) if f.endswith('.txt')]

        for txt_file in txt_files:
            file_path = os.path.join(extract_dir, txt_file)
            
            # Lê o arquivo .txt como DataFrame
            df = pd.read_csv(
                file_path,
                header=None,
                names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'],
                parse_dates=['Date']
            )
            
            # Extrai o ticker do nome do arquivo
            ticker = os.path.splitext(txt_file)[0].split('_')[0]
            df['ticker'] = ticker
            
            df_list.append(df)
            
            # Remove o .txt após uso para evitar acúmulo de arquivos
            os.remove(file_path)

# Junta tudo em um DataFrame só
full_df = pd.concat(df_list, ignore_index=True)

# Filtra e seleciona colunas desejadas
filtered_df = (
    full_df[full_df['Date'] > '2017-12-31']
    .loc[:, ['ticker', 'Date', 'Close', 'Volume']]
    .rename(columns={'Date': 'date', 'Close': 'close', 'Volume': 'volume'})
)

# Filtrar para datas maiores que 2012 e selecionar colunas desejadas
filtered_df = (
    full_df[full_df['Date'] > '2015-12-31']
    .loc[:, ['ticker', 'Date', 'Close', 'Volume']]
    .rename(columns={'Date': 'date', 'Close': 'close', 'Volume': 'volume'})
)

# Garante que a coluna 'date' está no formato datetime
filtered_df['date'] = pd.to_datetime(filtered_df['date'])

# Adiciona coluna com a segunda-feira da semana como identificador da semana
filtered_df['week_start'] = filtered_df['date'] - pd.to_timedelta(filtered_df['date'].dt.weekday, unit='d')

# Ordena por ticker e data
filtered_df = filtered_df.sort_values(['ticker', 'date'])

# Agrupa por ticker e semana
weekly_returns = (
    filtered_df
    .groupby(['ticker', 'week_start'])
    .agg(first_price=('close', 'first'), last_price=('close', 'last'))
    .dropna()  # garante que as duas existam
    .reset_index()
)

# Calcula o log return semanal
weekly_returns['log_return'] = np.log(weekly_returns['last_price'] / weekly_returns['first_price'])

# Extrai ano e mês da semana
weekly_returns['year'] = weekly_returns['week_start'].dt.year
weekly_returns['month'] = weekly_returns['week_start'].dt.month

# Cria a pasta (se não existir)
output_dir = 'C:\\Users\\msses\\Desktop\\ETF\\weekly_log_returns'
os.makedirs(output_dir, exist_ok=True)

# Agrupa e salva por (ano, mês)
for (year, month), group in weekly_returns.groupby(['year', 'month']):
    filename = f'weekly_log_returns_{year}_{month:02d}.csv'
    filepath = os.path.join(output_dir, filename)
    
    group.to_csv(filepath, index=False)